Distance to Nearest City	cit.017	http://forobs.jrc.ec.europa.eu/products/gam/

In [4]:
import numpy as np
import pandas as pd
import rasterio as rio

import boto3
import requests as req

from matplotlib import pyplot as plt
%matplotlib inline
import os
import sys
import threading

In [5]:
s3_upload = boto3.client("s3")
s3_download = boto3.resource("s3")

s3_bucket = "wri-public-data"
s3_folder = "resourcewatch/raster/cit_017_distance_to_nearest_city/"

s3_file = "cit_017_distance_to_nearest_city.tif"

s3_key_orig = s3_folder + s3_file
s3_key_edit = s3_key_orig[0:-4] + "_edit.tif"


class ProgressPercentage(object):
        def __init__(self, filename):
            self._filename = filename
            self._size = float(os.path.getsize(filename))
            self._seen_so_far = 0
            self._lock = threading.Lock()

        def __call__(self, bytes_amount):
            # To simplify we'll assume this is hooked up
            # to a single filename.
            with self._lock:
                self._seen_so_far += bytes_amount
                percentage = (self._seen_so_far / self._size) * 100
                sys.stdout.write("\r%s  %s / %s  (%.2f%%)"%(
                        self._filename, self._seen_so_far, self._size,
                        percentage))
                sys.stdout.flush()

In [ ]:
local_orig = "./temp.tif"

s3_download.meta.client.download_file(s3_bucket, s3_key_orig, local_orig)

In [ ]:
with rasterio.open(local_orig) as src:
    profile = src.profile
    data = src.read(1)

In [ ]:
print(profile)

In [ ]:
local_edit = "./temp_edit.tif"

In [ ]:
# Note - this is the core of Vizz's netcdf2tif function

with rio.open(local_orig, 'r') as src:
    # This assumes data is readable by rasterio
    # May need to open instead with netcdf4.Dataset, for example
   
    data = src.read()[0]
    
    rows = data.shape[0]
    columns = data.shape[1]
    
    print(rows)
    print(columns)
    
    # Latitude bounds
    south_lat = -90
    north_lat = 90

    # Longitude bounds
    west_lon = -180
    east_lon = 180
    
    transform = rasterio.transform.from_bounds(west_lon, south_lat, east_lon, north_lat, columns, rows)
    
    # Profile
    
    no_data_val = -1
    target_projection = 'EPSG:4326'
    target_data_type = np.int32
    
    profile = {
        'driver':'GTiff', 
        'height':rows, 
        'width':columns, 
        'count':1, 
        'dtype':target_data_type, 
        'crs':target_projection, 
        'transform':transform, 
        'compress':'lzw', 
        'nodata': no_data_val
    }
    
    with rio.open(local_edit, "w", **profile) as dst:
        dst.write(data.astype(profile["dtype"]), 1)

In [ ]:
# Edit
s3_upload.upload_file(local_edit, s3_bucket, s3_key_edit,
                         Callback=ProgressPercentage(local_edit))

In [ ]:
os.remove(local_orig)
os.remove(local_edit)

In [6]:
os.environ["s3_key"] = "s3://wri-public-data/" + s3_key_edit
os.environ["gs_key"] = "gs://resource-watch-public/" + s3_key_edit

!gsutil cp $s3_key $gs_key



Updates are available for some Cloud SDK components.  To install them,
please run:
  $ gcloud components update

Non-MD5 etag ("491538df2add31f2dc874919b071ab93-61") present for key <Key: wri-public-data,resourcewatch/raster/cit_017_distance_to_nearest_city/cit_017_distance_to_nearest_city_edit.tif>, data integrity checks are not possible.
Copying s3://wri-public-data/resourcewatch/raster/cit_017_distance_to_nearest_city/cit_017_distance_to_nearest_city_edit.tif [Content-Type=binary/octet-stream]...
- [1 files][482.4 MiB/482.4 MiB]    9.1 MiB/s                                   
Operation completed over 1 objects/482.4 MiB.                                    


In [8]:
os.environ["asset_id"] = "users/resourcewatch/cit_017_travel_time_major_cities"

!earthengine upload image --asset_id=$asset_id $gs_key

Started upload task with ID: R6GE7W3OEDZNGU2SADETHISC


In [ ]:
!earthengine acl set public $asset_id
os.environ["band_names"] = str(band_names)
!earthengine asset set -p '(string)band_names='"$band_names" $asset_id